In [2]:
from neo4j import GraphDatabase

In [3]:
# Neo4j Desktop에서 설정한 데이터베이스의 접속 정보를 입력합니다.
# 기본값은 bolt://localhost:7687이며, 사용자 이름과 비밀번호는 설정한 것으로 입력합니다.
uri = "bolt://localhost:7687" # <<!nav>>볼트<<!/nav>> 프로토콜을 사용하므로 "bolt://"로 시작합니다 [2, 4].
user = "neo4j" # 또는 사용자가 설정한 사용자 이름
password = "ekvmsp2384" # 사용자가 설정한 비밀번호


In [4]:

# Driver 객체를 생성합니다.
driver = GraphDatabase.driver(uri, auth=(user, password))

In [5]:
# DB 모두 삭제

# 모든 데이터를 삭제하는 Cypher 쿼리
cypher_delete_all = """
MATCH (a) optional match (a)-[r]-() delete a,r
"""

# 세션을 생성하여 삭제 쿼리 실행
with driver.session() as session:
  print("모든 데이터를 삭제합니다...")
  
  # execute_write 트랜잭션 안에서 실행
  session.execute_write(lambda tx: tx.run(cypher_delete_all))
  
  print("데이터 삭제 완료.")


모든 데이터를 삭제합니다...
데이터 삭제 완료.


In [6]:
bus_csv_file = './20251027_서울시_버스정류장_주소_거리.csv'

In [7]:
import pandas as pd

bus_df = pd.read_csv(bus_csv_file)

In [8]:
bus_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
0,100100124,0017,2,102000158,3252,신용산지하차도,126.963882,37.532590,서울특별시,용산구,한강로3가,858.0
1,100100124,0017,3,102000161,3255,용산역,126.965822,37.528869,서울특별시,용산구,한강로3가,172.0
2,100100124,0017,4,102000162,3256,용산푸르지오써밋,126.964759,37.527574,서울특별시,용산구,한강로3가,390.0
3,100100124,0017,5,102000029,3122,한강대교북단,126.963570,37.524968,서울특별시,용산구,한강로3가,567.0
4,100100124,0017,6,102000200,3294,서부이촌동입구,126.959497,37.523863,서울특별시,용산구,한강로3가,383.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41256,100000020,청와대A01,2,100000415,1119,영추문,126.973982,37.578930,서울특별시,종로구,세종로,450.0
41257,100000020,청와대A01,3,100000416,1601,청와대,126.973762,37.582971,서울특별시,종로구,세종로,533.0
41258,100000020,청와대A01,4,100000417,1602,춘추문,126.979657,37.583126,서울특별시,종로구,팔판동,472.0
41259,100000020,청와대A01,5,100000418,1603,경복궁.국립민속박물관,126.979590,37.579407,서울특별시,종로구,세종로,884.0


In [9]:
bus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41261 entries, 0 to 41260
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ROUTE_ID  41261 non-null  int64  
 1   노선명       41261 non-null  object 
 2   순번        41261 non-null  int64  
 3   NODE_ID   41261 non-null  int64  
 4   ARS_ID    41261 non-null  int64  
 5   정류소명      41261 non-null  object 
 6   X좌표       41261 non-null  float64
 7   Y좌표       41261 non-null  float64
 8   시         41261 non-null  object 
 9   구         41261 non-null  object 
 10  동         41261 non-null  object 
 11  거리        36019 non-null  float64
dtypes: float64(3), int64(4), object(5)
memory usage: 3.8+ MB


In [11]:
%%time
# 모든 데이터를 삭제하는 Cypher 쿼리
cypher_load_stop = """
// 1. 버스 정류장 노드 생성 (변경된 파일 이름 적용)
LOAD CSV WITH HEADERS FROM 'file:///20251027_bus.csv' AS row
MERGE (s:Station {id: row.NODE_ID})
ON CREATE SET
  s.name = row.정류소명,
  s.arsId = row.ARS_ID,
  s.location = point({latitude: toFloat(row.Y좌표), longitude: toFloat(row.X좌표)}),
  s.type = 'bus';
"""

# 4. 세션 생성 및 쿼리 실행 (execute_write 사용)
def load_cypher_load_stop(tx):
    # 쿼리 실행
    result = tx.run(cypher_load_stop)
    
    # !!! 수정된 부분: .consume()을 사용하여 결과를 소모하고 요약 정보를 가져옵니다.
    summary = result.consume()
    
    # 요약 정보에서 counters (카운터)를 추출합니다.
    return f"쿼리 실행 완료. 통계: {summary.counters}"
  


# 세션을 생성하여 삭제 쿼리 실행
with driver.session() as session:
  print("CSV 데이터 임포트를 시작합니다...")
    
  summary = session.execute_write(load_cypher_load_stop)
    
  print(f"데이터 임포트 완료.summary={summary}")



CSV 데이터 임포트를 시작합니다...
데이터 임포트 완료.summary=쿼리 실행 완료. 통계: SummaryCounters{labels_added: 12894, nodes_created: 12894, properties_set: 64470, contains_updates: True, contains_system_updates: False}
CPU times: user 5.1 ms, sys: 5.8 ms, total: 10.9 ms
Wall time: 21.8 s


In [12]:
%%time
# 모든 데이터를 삭제하는 Cypher 쿼리
cypher_load_bus = """
// 2. 버스 노선 노드 생성 (변경된 파일 이름 적용)
LOAD CSV WITH HEADERS FROM 'file:///20251027_bus.csv' AS row
MERGE (r:Route {id: row.ROUTE_ID})
ON CREATE SET
  r.name = row.노선명,
  r.type = 'bus';
"""

# 4. 세션 생성 및 쿼리 실행 (execute_write 사용)
def load_cypher_load_bus(tx):
    # 쿼리 실행
    result = tx.run(cypher_load_bus)
    
    # !!! 수정된 부분: .consume()을 사용하여 결과를 소모하고 요약 정보를 가져옵니다.
    summary = result.consume()
    
    # 요약 정보에서 counters (카운터)를 추출합니다.
    return f"쿼리 실행 완료. 통계: {summary.counters}"
  


# 세션을 생성하여 삭제 쿼리 실행
with driver.session() as session:
  print("CSV 데이터 임포트를 시작합니다...")
    
  summary = session.execute_write(load_cypher_load_bus)
    
  print(f"데이터 임포트 완료.summary={summary}")



CSV 데이터 임포트를 시작합니다...
데이터 임포트 완료.summary=쿼리 실행 완료. 통계: SummaryCounters{labels_added: 705, nodes_created: 705, properties_set: 2115, contains_updates: True, contains_system_updates: False}
CPU times: user 3.52 ms, sys: 5 ms, total: 8.52 ms
Wall time: 1.19 s


In [13]:
%%time
# 모든 데이터를 삭제하는 Cypher 쿼리
cypher_load_next = """
// 2. 버스 노선 노드 생성 (변경된 파일 이름 적용)
LOAD CSV WITH HEADERS FROM 'file:///20251027_bus.csv' AS row
// route.id로 Route 노드를 찾습니다.
MATCH (route:Route {id: row.ROUTE_ID})
// 노선(route)별로, 순번(row.순번)에 따라 정렬합니다.
WITH route, row ORDER BY toInteger(row.순번)

// 정렬된 'row' 객체 전체를 리스트로 수집합니다.
// (NODE_ID만 수집하면 '거리' 값에 접근할 수 없습니다.)
WITH route, collect(row) AS rows

// 리스트를 순회하며 (현재 정류장 row, 다음 정류장 row) 쌍을 만듭니다.
UNWIND range(0, size(rows) - 2) AS i
// 현재 row(row1)와 다음 row(row2)를 변수로 가져옵니다.
WITH route, rows[i] AS row1, rows[i+1] AS row2

// row1과 row2의 NODE_ID를 사용해 이미 생성된 Station 노드를 찾습니다.
MATCH (s1:Station {id: row1.NODE_ID})
MATCH (s2:Station {id: row2.NODE_ID})

// 관계를 생성합니다.
MERGE (s1)-[r:NEXT_STOP {routeName: route.name}]->(s2)
// SET을 사용해 'row1'의 '거리' 값을 숫자로 변환하여 'distance' 속성으로 추가합니다.
ON CREATE SET
  r.distance = toFloat(row1.거리)
ON MATCH SET
  r.distance = toFloat(row1.거리)
"""

# 4. 세션 생성 및 쿼리 실행 (execute_write 사용)
def load_cypher_load_next(tx):
    # 쿼리 실행
    result = tx.run(cypher_load_next)
    
    # !!! 수정된 부분: .consume()을 사용하여 결과를 소모하고 요약 정보를 가져옵니다.
    summary = result.consume()
    
    # 요약 정보에서 counters (카운터)를 추출합니다.
    return f"쿼리 실행 완료. 통계: {summary.counters}"
  


# 세션을 생성하여 삭제 쿼리 실행
with driver.session() as session:
  print("CSV 데이터 임포트를 시작합니다...")
    
  summary = session.execute_write(load_cypher_load_next)
    
  print(f"데이터 임포트 완료.summary={summary}")



CSV 데이터 임포트를 시작합니다...
데이터 임포트 완료.summary=쿼리 실행 완료. 통계: SummaryCounters{relationships_created: 40193, properties_set: 75507, contains_updates: True, contains_system_updates: False}
CPU times: user 9.03 ms, sys: 16.6 ms, total: 25.6 ms
Wall time: 2min 22s


In [260]:
bus_df[bus_df['노선명'] == '240']

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
7883,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,278.0
7884,100100407,240,2,106000441,7437,새우개마을입구,127.099763,37.614242,서울특별시,중랑구,신내동,944.0
7885,100100407,240,3,106000189,7284,봉화초등학교,127.091529,37.613877,서울특별시,중랑구,신내동,154.0
7886,100100407,240,4,106000190,7285,신내6단지아파트.옹기테마공원,127.090418,37.613897,서울특별시,중랑구,신내동,394.0
7887,100100407,240,5,106000193,7288,신내우체국.5단지두산대림아파트,127.090017,37.616566,서울특별시,중랑구,신내동,524.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7999,100100407,240,117,106000188,7283,봉화초등학교,127.091724,37.613761,서울특별시,중랑구,신내동,535.0
8000,100100407,240,118,106000186,7281,신내7단지아파트.홈플러스,127.093887,37.615253,서울특별시,중랑구,신내동,133.0
8001,100100407,240,119,106000257,7354,중랑소방서,127.095217,37.614680,서울특별시,중랑구,신내동,732.0
8002,100100407,240,120,106000241,7337,중랑공영차고지,127.102023,37.613645,서울특별시,중랑구,신내동,148.0


In [264]:
bus_df[bus_df['NODE_ID'] == 106000182]

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
1844,100100425,1122,46,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,352.0
5924,100100186,2012,10,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,394.0
6669,100100192,2113,51,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,394.0
6778,100100598,2115,14,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,352.0
7592,111000019,2236,18,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,394.0
7666,100100599,2311,6,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,352.0
7837,100100611,2312,60,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,394.0
7893,100100407,240,11,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,555.0
8021,100100595,241,18,106000182,7277,신내10단지아파트,127.095563,37.609147,서울특별시,중랑구,신내동,394.0


In [ ]:
bus_df[bus_df['노선명'] == '2012']

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
5915,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,317.0
5916,100100186,2012,2,106000437,7435,신내데시앙아파트후문,127.100263,37.613075,서울특별시,중랑구,신내동,178.0
5917,100100186,2012,3,106000438,7433,서울의료원후문,127.098397,37.613665,서울특별시,중랑구,신내동,313.0
5918,100100186,2012,4,106000250,7347,중랑소방서,127.095220,37.614875,서울특별시,중랑구,신내동,149.0
5919,100100186,2012,5,106000187,7282,신내7단지아파트.홈플러스,127.093706,37.615524,서울특별시,중랑구,신내동,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6023,100100186,2012,109,106000257,7354,중랑소방서,127.095217,37.614680,서울특별시,중랑구,신내동,220.0
6024,100100186,2012,110,106000439,7432,서울의료원후문,127.097435,37.613773,서울특별시,중랑구,신내동,188.0
6025,100100186,2012,111,106000436,7434,신내데시앙아파트후문,127.099442,37.613160,서울특별시,중랑구,신내동,323.0
6026,100100186,2012,112,106000241,7337,중랑공영차고지,127.102023,37.613645,서울특별시,중랑구,신내동,148.0


In [4]:
from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

try:
  driver.verify_connectivity()
  print("Neo4j 연결 성공")

  # 2. 생성할 인덱스 Cypher 쿼리
  # (IF NOT EXISTS: 이미 존재하면 오류 없이 건너뜁니다)
  
  # 2-1. Station(id) 인덱스: 경로 탐색 시 시작/종료 노드 검색 속도 향상 (필수)
  cypher_create_station_index = """
  CREATE INDEX station_id_index IF NOT EXISTS 
  FOR (s:Station) 
  ON (s.id)
  """
  
  # 2-2. Route(id) 인덱스: LOAD CSV로 관계 생성 시 Route 노드 검색 속도 향상 (권장)
  cypher_create_route_index = """
  CREATE INDEX route_id_index IF NOT EXISTS 
  FOR (r:Route) 
  ON (r.id)
  """
  
  # 2-3. (보너스) Station(location) Point 인덱스: 지리공간 검색 속도 향상
  cypher_create_point_index = """
  CREATE POINT INDEX station_location_index IF NOT EXISTS
  FOR (s:Station)
  ON (s.location)
  """

  # 3. 세션을 열고 인덱스 생성 쿼리 실행
  with driver.session() as session:
    print("Station(id) 인덱스를 생성합니다...")
    session.run(cypher_create_station_index)
    
    print("Route(id) 인덱스를 생성합니다...")
    session.run(cypher_create_route_index)
    
    print("Station(location) Point 인덱스를 생성합니다...")
    session.run(cypher_create_point_index)
    
    print("인덱스 작업 완료 (이미 존재하면 건너뜀).")

except ServiceUnavailable as e:
  print(f"Neo4j 데이터베이스에 연결할 수 없습니다: {e}")
except Exception as e:
  print(f"오류 발생: {e}")


Neo4j 연결 성공
Station(id) 인덱스를 생성합니다...
Route(id) 인덱스를 생성합니다...
Station(location) Point 인덱스를 생성합니다...
인덱스 작업 완료 (이미 존재하면 건너뜀).


In [ ]:
# 240번 버스의 시점-종점 테스트


cypher_find_path = """
// 1. 시작 노드와 종료 노드를 (인덱스를 사용해) 빠르게 찾습니다.
MATCH (start:Station {id: $start_node_id}), (end:Station {id: $end_node_id})

// 2. 두 노드 사이의 경로를 찾습니다.
MATCH path = (start)-[r:NEXT_STOP*]->(end)

// 3. 경로상의 거리(distance) 합계를 계산합니다.
WITH path, reduce(totalDist = 0.0, rel IN relationships(path) | totalDist + rel.distance) AS totalDistance

// 4. 경로와 총 거리를 반환합니다.
RETURN path, totalDistance

// 5. 총 거리가 가장 짧은 순으로 정렬합니다.
ORDER BY totalDistance ASC

// 6. 가장 짧은 경로 1개만 가져옵니다.
LIMIT 1
"""

# 3. 경로를 찾을 시작점과 도착점 ID (예시)
start_station_id = "106000182" # 예: 서울역
end_station_id = "106000175"   # 예: 시청역

# 4. 세션을 열고 읽기 트랜잭션으로 쿼리 실행
try:
  with driver.session() as session:
    print(f"경로 탐색 중: {start_station_id} -> {end_station_id}")
    
    result = session.execute_read(
      lambda tx: tx.run(cypher_find_path, 
                        start_node_id=start_station_id, 
                        end_node_id=end_station_id).data()
    )

    # 5. 결과 처리
    if not result:
      print("경로를 찾을 수 없습니다.")
    else:
      # LIMIT 1이므로 결과는 1개
      record = result[0]
      path = record['path']
      total_distance = record['totalDistance']
      
      print(f"--- 최단 경로 발견 ---")
      print(f"총 거리: {total_distance:.2f} 미터")
      
      print("\n--- 경로 상세 ---")
      # path.nodes는 경로상의 노드 리스트입니다.
      station_names = [node['name'] for node in path.nodes]
      print(" -> ".join(station_names))
      
      # (선택) 경로의 관계(구간) 상세 정보 출력
      # print("\n--- 구간별 상세 ---")
      # for rel in path.relationships:
      #   start_node = rel.start_node
      #   end_node = rel.end_node
      #   print(f"  {start_node['name']} -> {end_node['name']} "
      #         f"({rel['routeName']} / {rel['distance']:.2f}m)")

except Exception as e:
  print(f"오류 발생: {e}")

finally:
  # 6. Driver 닫기
  driver.close()

경로 탐색 중: 106000182 -> 106000175


In [ ]:
# 240의 1번에서 출발 11번에서 2012번으로 환승 하여 20번에서 하차 테스트 

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=uri, 
    username=user,
    password=password,
)

/var/folders/sj/3r49bs6178v7nfhcxh7lpzd80000gn/T/ipykernel_39864/2951053509.py:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [ ]:
import os
# OpenAI 관련 클래스를 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain.chains import GraphCypherQAChain
# 이미 정의된 graph 객체를 사용합니다. (Neo4jGraph 인스턴스)

# (필수) OPENAI_API_KEY 환경 변수가 설정되어 있어야 합니다.
os.environ["OPENAI_API_KEY"] = "KEY_VALUE"

# Neo4j 스키마를 새로 고칩니다. (APOC 오류 해결 후 정상 작동한다고 가정)
graph.refresh_schema()

# OpenAI 모델 인스턴스 생성
# Cypher 생성 및 최종 답변 생성에 모두 Claude 3.5 Sonnet과 비슷한 수준의 모델을 사용합니다.
# gpt-4o 또는 gpt-4-turbo가 복잡한 Cypher 생성에 적합합니다.
openai_llm = ChatOpenAI(
    model="gpt-4.1",  # 최신 고성능 모델 사용 권장
    temperature=0  # Cypher 생성 시 창의성보다 정확성이 중요하므로 0을 사용합니다.
)

# GraphCypherQAChain 구성
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=openai_llm,  # Cypher 쿼리 생성에 OpenAI 모델 사용
    qa_llm=openai_llm,      # 최종 답변 생성에 OpenAI 모델 사용
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
)

print("✅ GraphCypherQAChain이 OpenAI 모델(gpt-4o)로 성공적으로 구성되었습니다.")
# 이제 cypher_chain.invoke({"query": "질문 내용"}) 형태로 사용할 수 있습니다.

✅ GraphCypherQAChain이 OpenAI 모델(gpt-4o)로 성공적으로 구성되었습니다.


In [ ]:
cypher_chain.invoke(
    {"query": "Station의 id가 106000319 에서 시작해서 106000175 로 끝나는 NEXT_STOP 리스트를 조회해줘"}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH path = (start:Station {id: '106000319'})-[:NEXT_STOP*]->(end:Station {id: '106000175'})
RETURN path


KeyboardInterrupt: 

In [155]:
cypher_chain.invoke(
    {"query": "관심사가 같은 사람끼리 커피챗을 짝지어주려 합니다. 모든 사람이 포함되도록 그룹을 짝지어주세요."}
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d1:개발자)-[:작성하다]->(:블로그포스트)-[:주제이다]->(t:주제)<-[:주제이다]-(:블로그포스트)<-[:작성하다]-(d2:개발자)
WHERE d1 <> d2
WITH DISTINCT 
  CASE WHEN d1.이름 < d2.이름 THEN [d1.이름, d2.이름] ELSE [d2.이름, d1.이름] END AS pair, t.이름 AS 주제
RETURN pair[0] AS 개발자1, pair[1] AS 개발자2, collect(DISTINCT 주제) AS 공통주제
ORDER BY 개발자1, 개발자2
Full Context:
[{'개발자1': '김민지', '개발자2': '김철수', '공통주제': ['백엔드 개발', '프론트엔드 개발', '모바일 개발']}, {'개발자1': '김민지', '개발자2': '박영희', '공통주제': ['백엔드 개발', '프론트엔드 개발']}, {'개발자1': '김민지', '개발자2': '이상훈', '공통주제': ['프론트엔드 개발']}, {'개발자1': '김민지', '개발자2': '이지훈', '공통주제': ['백엔드 개발']}, {'개발자1': '김민지', '개발자2': '정미경', '공통주제': ['백엔드 개발']}, {'개발자1': '김민지', '개발자2': '최유진', '공통주제': ['백엔드 개발', '프론트엔드 개발', '모바일 개발']}, {'개발자1': '김민지', '개발자2': '홍길동', '공통주제': ['프론트엔드 개발']}, {'개발자1': '김철수', '개발자2': '이상훈', '공통주제': ['백엔드 개발']}, {'개발자1': '김철수', '개발자2': '이지훈', '공통주제': ['백엔드 개발', '모바일 개발']}, {'개발자1': '김철수', '개발자2': '정미경', '공통주제': ['백엔드 개발']}]

> Finished chain.


{'query': '관심사가 같은 사람끼리 커피챗을 짝지어주려 합니다. 모든 사람이 포함되도록 그룹을 짝지어주세요.',
 'result': '김민지-김철수, 김민지-박영희, 김민지-이상훈, 김민지-이지훈, 김민지-정미경, 김민지-최유진, 김민지-홍길동, 김철수-이상훈, 김철수-이지훈, 김철수-정미경이 관심사가 같아 커피챗을 짝지을 수 있습니다. 모든 사람이 포함됩니다.'}

In [158]:
import pandas as pd

In [159]:
food_store_df = pd.read_csv('bluer_foodV2.csv')

In [160]:
food_store_df

,id,상세URL,가게,주소,카테고리,리본개수,레드리본 선정,서울 2025 선정,라벨,이미지URL,이미지파일,소개
0,30639,https://www.bluer.co.kr/restaurants/30639,다이닝마,서울특별시 강남구 언주로152길 8 (신사동) 유일빌딩 2층,일반중식,3,N,Y,서울 2025 선정,https://www.bluer.co.kr/images/es_f1fa131ef832...,images\30639_다이닝마.png,"중국 3대 진미로 뽑히는 샥스핀을 전문으로 하는 고급 중식당으로, 룸으로만 구성되어..."
1,29083,https://www.bluer.co.kr/restaurants/29083,쵸이닷,서울특별시 강남구 도산대로 457 (청담동) 앙스돔빌딩 3층,"이노베이티브퀴진, 분자요리, 이탈리아식",3,Y,Y,레드리본 선정 서울 2025 선정,https://www.bluer.co.kr/images/es_d0546f2b0dd7...,images\29083_쵸이닷.jpg,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...
2,4953,https://www.bluer.co.kr/restaurants/4953,비스트로드욘트빌,서울특별시 강남구 선릉로158길 13-7 (청담동) 이안빌딩 1층,프랑스식,3,N,Y,서울 2025 선정,https://www.bluer.co.kr/images/es_5a2c60f575f7...,images\4953_비스트로드욘트빌.jpg,편안한 분위기에서 와인과 프랑스 음식을 즐길 수 있는 곳. 클래식한 정통 프렌치 비...
3,27144,https://www.bluer.co.kr/restaurants/27144,밍글스,서울특별시 강남구 도산대로67길 19 (청담동) 힐탑빌딩 2층,뉴코리안,3,N,Y,서울 2025 선정,https://www.bluer.co.kr/images/es_864204db95a1...,images\27144_밍글스.jpg,국내외에서 이름을 드높이고 있는 강민구 셰프의 뉴코리안 레스토랑. 사찰음식과 한식 ...
4,254,https://www.bluer.co.kr/restaurants/254,라미띠에,서울특별시 강남구 도산대로67길 30 (청담동) 2층,프랑스식,3,N,Y,서울 2025 선정,https://www.bluer.co.kr/images/es_fbc959011200...,images\254_라미띠에.jpg,서울의 프렌치 레스토랑을 얘기할 때 빼놓을 수 없는 곳. 장명식 셰프가 클래식한 스...
...,...,...,...,...,...,...,...,...,...,...,...,...
7709,34368,https://www.bluer.co.kr/restaurants/34368,로익스커피,서울특별시 마포구 월드컵로29길 37-15 (망원동),커피전문점,0,N,N,NaN,https://www.bluer.co.kr/images/es_cce257e37d0d...,images\34368_로익스커피.jpg,유종규 바리스타가 운영하는 커피 매장 겸 커피 교육장. 직접 볶은 스페셜티 원두로 ...
7710,33452,https://www.bluer.co.kr/restaurants/33452,커퍼시티,서울특별시 마포구 동교로 48 (합정동),커피전문점,0,N,N,NaN,https://www.bluer.co.kr/assets/image/restauran...,images\33452_커퍼시티.png,필터커피를 주로 하는 스페셜티 커피 전문점. 검증된 브랜드에서 로스팅한 퀄리티 좋은...
7711,40269,https://www.bluer.co.kr/restaurants/40269,리틀빅토리,서울특별시 마포구 연희로1길 41 (연남동) 예일빌라 102호,"구움과자, 베이커리",0,N,N,NaN,https://www.bluer.co.kr/assets/image/restauran...,images\40269_리틀빅토리.png,"다양한 종류의 구움과자를 선보이는 곳. 마카롱, 카늘레, 사브레 브루통, 티그레, ..."
7712,40133,https://www.bluer.co.kr/restaurants/40133,티브커피하우스,"서울특별시 마포구 동교로51안길 11 (연남동) 1, 2층",카페,0,N,N,NaN,https://www.bluer.co.kr/images/es_67f6599e9b41...,images\40133_티브커피하우스.jpg,오래된 주택의 구조를 그대로 살려 새롭게 꾸민 카페. 커피인굿스피릿 챔피언 하청비 ...
